In [219]:
%run classesDefinations.ipynb

In [220]:
CAPACITY = 10
BUS_ORIGIN = Cords(35.8942679, 14.5086503) #CORDS OF THE 15 VALLETTA BUS BAY)
BUS_STOP_DATA_PATH = "./Data/BusStopsMalta/export.json"

In [221]:
from datetime import datetime, timedelta
import json
import math
import random

def divide_time_interval(n):
    # Define start and end times
    start_time = datetime.strptime("08:00 AM", "%I:%M %p")
    end_time = datetime.strptime("12:00 AM", "%I:%M %p")

    # Calculate total duration in seconds
    total_duration = (end_time - start_time).seconds

    # Calculate duration of each interval in seconds
    interval_duration = total_duration / n

    # Generate time intervals
    intervals = []
    for i in range(n):
        interval_start = start_time + timedelta(seconds=i * interval_duration)
        intervals.append(interval_start.time())

    return intervals

def getTimeDifference(time1, time2):
    today = datetime.now().date()
    datetime1 = datetime.combine(today, time1)
    datetime2 = datetime.combine(today, time2)

    difference = datetime1 - datetime2
    return difference.seconds

In [222]:
class BusHandler():
    def __init__(self, vehicleAmount, numberOfRequests):
        self.vehicles = self.initVehiles(vehicleAmount)
        self.requests = self.initRequests(numberOfRequests)
        self.currentRequest = self.requests[0]
        self.currentRequestIndex = 0
        self.rejectedRequests = []

    def __str__ (self):
        stringOutput = "==========================================\n Vehicles: \n"
        for vehile in self.vehicles:
            stringOutput += str(vehile) + "\n"
        stringOutput += "==========================================\n Requests: \n"
        for request in self.requests:
            stringOutput += str(request) + "\n"
        stringOutput += "==========================================\n"
        return stringOutput

    def initVehiles(self, vehicleAmount):
        vehicles = []
        for i in range(vehicleAmount):
            vehicles.append(Vehicle(i, CAPACITY, BUS_ORIGIN))
        return vehicles

    def initRequests(self, numberOfRequests):
        requestId = 0
        timeIntervals = divide_time_interval(numberOfRequests)
        f = open(BUS_STOP_DATA_PATH, "r")
        data = json.load(f)
        elements = data["elements"]
        elementsLen = len(elements)
        requestList = []
        for i in range(numberOfRequests):
            #Get two random indexes from all the bus stops 
            r1 = random.randint(0, elementsLen-1)
            r2 = random.randint(0, elementsLen-1)
            while r1 == r2:
                r2 = random.randint(0, elementsLen-1)
            
            #Get the coordinates of the two bus stops
            lat1 = elements[r1]["lat"]
            lon1 = elements[r1]["lon"]
            lat2 = elements[r2]["lat"]
            lon2 = elements[r2]["lon"]

            #Create the request Cords
            reqPickup = Cords(lat1, lon1)
            reqDropoff = Cords(lat2, lon2)

            #Create the request
            requestList.append(Request(requestId, reqPickup, reqDropoff, timeIntervals[i],random.randint(1, 3)))

            #Increment the requestId
            requestId += 1
        f.close()
        return requestList
    
    def endCheck(self):
        if self.currentRequestIndex < len(self.requests):
            self.currentRequest = self.requests[self.currentRequestIndex]
            return False
        else:
            return True
    
    def rejectRequest(self):
        #Error Checking
        if(self.currentRequestIndex >= len(self.requests)):
            raise ValueError("No more requests to reject, index exceeded the requests list length.")
        #Reject the current request and move to the next one
        self.rejectedRequests.append(self.currentRequest)
        self.currentRequestIndex += 1
        return self.endCheck()
        
    def acceptRequest(self, vehicleIndex):
        #Error Checking
        if(vehicleIndex >= len(self.vehicles)):
            raise ValueError("Vehicle index exceeded the vehicles list length.")

        #Accept the current request and move to the next one
        self.vehicles[vehicleIndex].addRequestToRoute(self.currentRequest)
        self.currentRequestIndex += 1
        return self.endCheck()
        
    def getCurrentRequest(self):
        return self.currentRequest
    
    def updateState(self):
        timeDiffernce = getTimeDifference(self.currentRequest.getTime(),self.requests[self.currentRequestIndex-1].getTime())
        for vehicle in self.vehicles:
            vehicle.move(timeDiffernce)

In [223]:
busHandler = BusHandler(5, 50)
print(busHandler)

 Vehicles: 
Vehicle: id: 0, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 1, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 2, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 3, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 4, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
 Requests: 
Request: id: 0, passengerAmount: 1, time: 08:00:00, origin: Req_Id: 0, lat: 35.9010746, long: 14.4482768, True, destination: Req_Id: 0, lat: 35.8474985, long: 14.5264874, False,
Request: id: 1, passengerAmount: 2, time: 08:19:12, origin: Req_Id: 1, lat: 35.9234336, long: 14.4776733, True, destination: Req_Id: 1, lat: 35.8934338, long: 14.4355813, False,
Request: id: 2, passengerAmount: 1, time: 08:38:24, origin: Req_Id: 2, lat: 35.9087176, long: 1

In [224]:
busHandler.acceptRequest(0)
busHandler.updateState()
print(busHandler)

Added Request id:  0
Vehicle  0  is moving
Completed Legs  1 Route Time  435.80000000000007
Completed Legs  1 Len Legs  2
Time Difference  1152
716.1999999999999
Completed Legs  1 Completed Steps  16 Time Left  196.69999999999987 Bearing  182
 Vehicles: 
Vehicle: id: 0, currentPassengers: 1, route_size: 1, currentPosition: (lat: 35.85249552128534,long: 14.494195145140802)
Vehicle: id: 1, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 2, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 3, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 4, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
 Requests: 
Request: id: 0, passengerAmount: 1, time: 08:00:00, origin: Req_Id: 0, lat: 35.9010746, long: 14.4482768, True, destination: Req_Id: 0, lat: 35.8474985, long: 14.5264874, False,
Request: id: 1,

In [225]:
busHandler.acceptRequest(0)
busHandler.updateState()
print(busHandler)

Added Request id:  1
Vehicle  0  is moving
Completed Legs  1 Route Time  423.6
Completed Legs  1 Len Legs  3
Time Difference  1152
728.4
Completed Legs  1 Completed Steps  8 Time Left  306.59999999999997 Bearing  296
 Vehicles: 
Vehicle: id: 0, currentPassengers: 2, route_size: 2, currentPosition: (lat: 35.9043324637241,long: 14.428190933729912)
Vehicle: id: 1, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 2, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 3, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 4, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
 Requests: 
Request: id: 0, passengerAmount: 1, time: 08:00:00, origin: Req_Id: 0, lat: 35.9010746, long: 14.4482768, True, destination: Req_Id: 0, lat: 35.8474985, long: 14.5264874, False,
Request: id: 1, passengerAmount: 2, time: 

In [226]:
busHandler.acceptRequest(0)
busHandler.updateState()
print(busHandler)

Added Request id:  2
Vehicle  0  is moving
Completed Legs  2 Route Time  1093.7000000000003
Completed Legs  2 Len Legs  4
Time Difference  1152
58.29999999999973
Completed Legs  2 Completed Steps  2 Time Left  18.699999999999726 Bearing  358
 Vehicles: 
Vehicle: id: 0, currentPassengers: 1, route_size: 2, currentPosition: (lat: 35.896209432991206,long: 14.434331623428163)
Vehicle: id: 1, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 2, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 3, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
Vehicle: id: 4, currentPassengers: 0, route_size: 0, currentPosition: (lat: 35.8942679,long: 14.5086503)
 Requests: 
Request: id: 0, passengerAmount: 1, time: 08:00:00, origin: Req_Id: 0, lat: 35.9010746, long: 14.4482768, True, destination: Req_Id: 0, lat: 35.8474985, long: 14.5264874, False,
Request: id: 1,